In [36]:
from frost2df import frost2df, obs2df, help, obsthin
from datetime import datetime, timedelta
from dateutil import parser, tz
import pandas as pd
import numpy as np
import requests
import json

In [19]:
elements = [
        'air_temperature',
        'surface_snow_thickness',
        'wind_speed',
        'wind_from_direction',
        'sum(precipitation_amount PT10M)'
]

In [21]:
fra_frost = frost_api('33950', 5, elements)

https://frost.met.no/observations/v0.jsonld?sources=SN33950&elements=air_temperature%2Csurface_snow_thickness%2Cwind_speed%2Cwind_from_direction%2Csum%28precipitation_amount+PT10M%29&referencetime=2024-11-17T22%3A00%3A00%2F2024-11-22T22%3A00%3A00&timeoffsets=PT0H


In [22]:
print(fra_frost)

       sourceId                        elementId             referenceTime  \
0     SN33950:0                  air_temperature 2024-11-17 22:00:00+00:00   
1     SN33950:0                  air_temperature 2024-11-17 22:00:00+00:00   
2     SN33950:0                       wind_speed 2024-11-17 22:00:00+00:00   
3     SN33950:0                       wind_speed 2024-11-17 22:00:00+00:00   
4     SN33950:0              wind_from_direction 2024-11-17 22:00:00+00:00   
...         ...                              ...                       ...   
3213  SN33950:0                       wind_speed 2024-11-22 21:00:00+00:00   
3214  SN33950:0                       wind_speed 2024-11-22 21:00:00+00:00   
3215  SN33950:0              wind_from_direction 2024-11-22 21:00:00+00:00   
3216  SN33950:0              wind_from_direction 2024-11-22 21:00:00+00:00   
3217  SN33950:0  sum(precipitation_amount PT10M) 2024-11-22 21:00:00+00:00   

      value     unit  qualityCode timeOffset timeResolution  le

In [4]:
def frost_api(stasjonsid, dager_tidligere, elements, timeoffsets='PT0H'):
    """
    Denne funksjonen henter værdata fra Frost API for en gitt stasjon og tidsperiode.
    Den konverterer svaret til en pandas DataFrame, setter 'referenceTime' som indeksen, sorterer DataFrame etter indeksen,
    og konverterer 'value' kolonnen til numerisk.

    Parametere:
    stasjonsid (str): IDen til stasjonen som dataene skal hentes fra.
    dager_tidligere (int): Antall dager tilbake i tid for å hente data.
    elements (str eller liste): Værelementene som skal hentes. Hvis en liste er gitt, blir den konvertert til en kommaseparert streng.
    timeoffsets (str, valgfritt): Tidsforskyvningene som skal brukes. Standard er 'PT0H'.

    Returnerer:
    df (DataFrame): En DataFrame som inneholder de hentede værdataene.
    """
    now = datetime.now().replace(minute=0, second=0, microsecond=0)

    #Finner tidspunkt for xx dager siden
    earlier_date = now - timedelta(days=dager_tidligere)

    # Konverterer til string
    now_str = now.isoformat()
    earlier_date_str = earlier_date.isoformat()

    # Hvis elements er en liste, altså fleire værelementer, konverterer til string
    if type(elements) == list:
        elements = ','.join(elements)

    # Lager parameterene som skal sendes til Frost API
    parameters = {
    'sources':'SN' + str(stasjonsid),
    'elements': elements,
    'referencetime': earlier_date_str + '/' + now_str,
    'timeoffsets': timeoffsets
    }

    # Henter data fra Frost API med hjelp av frost2df pakken
    df = obs2df(parameters=parameters, verbose=True)
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    return df



def bearbeid_frost(df):
    """
    Denne funksjonen resampler data til timeverdier for bedre visualisering. 
    Hvis elementet er lufttemperatur, snødybde, vindhastighet eller vindretning, beregnes gjennomsnittet for hver time.
    Hvis elementet er nedbør, summeres nedbøren for hver time.
    Funksjonen returnerer en DataFrame med resamplede verdier for hvert element.

    Parametere:
    df (DataFrame): DataFrame som inneholder data hentet fra Frost API.

    Returnerer:
    df_resampled (DataFrame): DataFrame med resamplede verdier for hvert element.
    """

    # Define the resampling operation for each element
    resampling_operations = {
        'air_temperature': 'mean',
        'surface_snow_thickness': 'mean',
        'wind_speed': 'mean',
        'wind_from_direction': 'mean',
        'sum(precipitation_amount PT10M)': 'sum'
    }

    # Group by 'elementId' and 'referenceTime', then resample and apply the operation for each group
    df_resampled = df.groupby('elementId').resample('H', on='referenceTime').agg({
        'value': lambda group: group.agg(resampling_operations[group.name])
    })
    # Reset the index and replace NaN with None
    df_resampled.reset_index(inplace=True)
    df_resampled['value'] = df_resampled['value'].replace({np.nan: None})

    return df_resampled



def frost_samledf(df):
    df_pivot = df.pivot(index='referenceTime', columns='elementId', values='value')
    return df_pivot


def assign_direction_to_bin(value):
    if value >= 337.5 or value < 22.5:
        return 'N'
    elif value < 67.5:
        return 'NØ'
    elif value < 112.5:
        return 'Ø'
    elif value < 157.5:
        return 'SØ'
    elif value < 202.5:
        return 'S'
    elif value < 247.5:
        return 'SV'
    elif value < 292.5:
        return 'V'
    elif value < 337.5:
        return 'NV'

def vindrose(stasjonsid, dager_tidligere):
    """
    Denne funksjonen henter vindhastighet og vindretning fra Frost API for en gitt stasjon og tidsperiode.
    Den kombinerer deretter disse dataene i en enkelt DataFrame, og kategoriserer vindhastigheten og vindretningen i binner.
    Til slutt, den returnerer en pivotert DataFrame som viser frekvensen av vindhastighet og retning kombinasjoner.

    Parametere:
    stasjonsid (str): IDen til stasjonen som dataene skal hentes fra.
    dager_tidligere (int): Antall dager tilbake i tid for å hente data.

    Returnerer:
    pivot_df (DataFrame): En DataFrame som viser frekvensen av vindhastighet og retning kombinasjoner.
    """
    direction_order = ['N', 'NØ', 'Ø', 'SØ', 'S', 'SV', 'V', 'NV']

    df_wind_speed = frost_api(stasjonsid, dager_tidligere, 'wind_speed', timeoffsets='PT0H')
    df_wind_from_direction = frost_api(stasjonsid, dager_tidligere, 'wind_from_direction', timeoffsets='PT0H')
    df_wind_speed = df_wind_speed.rename(columns={'value': 'wind_speed'})
    df_wind_from_direction = df_wind_from_direction.rename(columns={'value': 'wind_from_direction'})

    df_combined = df_wind_speed[['wind_speed']].merge(df_wind_from_direction[['wind_from_direction']], left_index=True, right_index=True)


    speed_bins = [0, 4, 8, 11, 14, 17, 20, 25, float('inf')]
    speed_labels = ['0-4 m/s', '4-8 m/s', '8-11 m/s', '11-14 m/s', '14-17 m/s', '17-20 m/s', '20-25 m/s', '>25 m/s']

    df_combined['direction_bin'] = df_combined['wind_from_direction'].apply(assign_direction_to_bin)
    df_combined['speed_bin'] = pd.cut(df_combined['wind_speed'], bins=speed_bins, labels=speed_labels, right=True)

    frequency_2d_df = df_combined.groupby(['direction_bin', 'speed_bin']).size().reset_index(name='Frequency')

  # Pivot the table to have wind speeds as columns and directions as rows
    pivot_df = frequency_2d_df.pivot(index='direction_bin', columns='speed_bin', values='Frequency')

    # Replace NaNs with 0s and ensure all direction bins are present
    pivot_df = pivot_df.reindex(direction_order, fill_value=0)

    # Convert the index to a categorical with the specified order
    pivot_df.index = pd.CategoricalIndex(pivot_df.index, categories=direction_order, ordered=True)

    # Sorting by index is not needed since reindexing has already ordered the index as per `direction_order`

    # Printing the pivot_df for verification
    print(pivot_df)

    return pivot_df

    

In [5]:
fra_frost = frost_api('58703', 5, elements)

https://frost.met.no/observations/v0.jsonld?sources=SN58703&elements=air_temperature%2Csurface_snow_thickness%2Cwind_speed%2Cwind_from_direction%2Csum%28precipitation_amount+PT10M%29&referencetime=2024-11-17T18%3A00%3A00%2F2024-11-22T18%3A00%3A00&timeoffsets=PT0H


In [23]:
client_id = 'b8b1793b-27ff-4f4d-a081-fcbcc5065b53'
client_secret = '7f24c0ca-ca82-4ed6-afcd-23e657c2e78c'


In [31]:
r = requests.get(f'https://frost.met.no/sources/v0.jsonld?ids=SN18700,SN50500,SN100&types=SensorSystem', auth=(client_id,client_secret))

In [38]:
#print(json.dumps(r.json(), indent=2))

In [53]:
def finn_element_ids(json_data):
    """
    Finner og printer ut alle unike elementId-verdier fra JSON-data.

    Args:
        json_data (dict): JSON-datastruktur.
    """
    # Hent 'data'-nøkkelen fra JSON-strukturen
    data = json_data.get("data", [])
    
    # Samle unike elementId-verdier
    element_ids = {item.get("elementId") for item in data if "elementId" in item}
    
    # Print unike elementId-verdier
    print("Unike elementId-verdier:")
    for element_id in sorted(element_ids):
        print(f"- {element_id}")

def sjekk_element_ids(json_data, elements):
    """
    Sjekker om noen elementId-verdier fra JSON-data finnes i en liste av elementer.

    Args:
        json_data (dict): JSON-datastruktur.
        elements (list): Liste over elementer som skal sjekkes mot.

    Returns:
        list: Liste over elementId-verdier som finnes i både JSON-data og elementlisten.
    """
    # Hent 'data'-nøkkelen fra JSON-strukturen
    data = json_data.get("data", [])
    
    # Samle elementId-verdier fra JSON-data
    element_ids = {item.get("elementId") for item in data if "elementId" in item}
    
    # Finn elementId-verdier som finnes i begge lister
    felles_elementer = [eid for eid in element_ids if any(eid.startswith(elem) for elem in elements)]
    
    return felles_elementer
    
elements = [
        'air_temperature',
        'surface_snow_thickness',
        'wind_speed',
        'wind_from_direction',
        'sum(precipitation_amount PT10M)'
]

In [57]:
r = requests.get('https://frost.met.no/observations/availableTimeSeries/v0.jsonld?sources=SN46430&referencetime=2024-11-23',auth=(client_id,client_secret))  

In [58]:
sjekk_element_ids(r.json(), elements)
#finn_element_ids(r.json())
#print(json.dumps(r.json(), indent=2))

['air_temperature',
 'surface_snow_thickness',
 'sum(precipitation_amount PT10M)']

In [61]:
r = requests.get('https://frost.met.no/sources/v0.jsonld?ids=SN18700&types=SensorSystem',auth=(client_id,client_secret)) 
print(json.dumps(r.json(), indent=2))

{
  "@context": "https://frost.met.no/schema",
  "@type": "SourceResponse",
  "apiVersion": "v0",
  "license": "https://creativecommons.org/licenses/by/3.0/no/",
  "createdAt": "2024-11-24T10:22:26Z",
  "queryTime": 1.423,
  "currentItemCount": 1,
  "itemsPerPage": 1,
  "offset": 0,
  "totalItemCount": 1,
  "currentLink": "https://frost.met.no/sources/v0.jsonld?ids=SN18700&types=SensorSystem",
  "data": [
    {
      "@type": "SensorSystem",
      "id": "SN18700",
      "name": "OSLO - BLINDERN",
      "shortName": "Oslo (Blindern)",
      "country": "Norge",
      "countryCode": "NO",
      "wmoId": 1492,
      "geometry": {
        "@type": "Point",
        "coordinates": [
          10.72,
          59.9423
        ],
        "nearest": false
      },
      "masl": 94,
      "validFrom": "1931-01-01T00:00:00.000Z",
      "county": "OSLO",
      "countyId": 3,
      "municipality": "OSLO",
      "municipalityId": 301,
      "ontologyId": 0,
      "stationHolders": [
        "MET.NO"


In [70]:
def hent_stasjonsdata(json_data):
    """
    Ekstraherer spesifikke felt fra stasjonsdata i JSON-format.

    Args:
        json_data (dict): JSON-data fra API-et.

    Returns:
        dict: Et dictionary med ønskede felt.
    """
    # Sjekk om det er data i JSON-strukturen
    data = json_data.get("data", [])
    if not data:
        return None  # Returner None hvis det ikke er data

    # Hent første element (hvis flere elementer finnes, kan det utvides)
    stasjon = data[0]

    # Ekstraher ønskede felt
    stasjonsinfo = {
        "name": stasjon.get("name"),
        "shortName": stasjon.get("shortName"),
        "masl": stasjon.get("masl"),
        "stationHolder": stasjon.get("stationHolders", []),
        "coordinates": stasjon.get("geometry", {}).get("coordinates", [])
    }

    return stasjonsinfo

In [71]:
print(hent_stasjonsdata(r.json()))

{'name': 'OSLO - BLINDERN', 'shortName': 'Oslo (Blindern)', 'masl': 94, 'stationHolder': ['MET.NO'], 'coordinates': [10.72, 59.9423]}


In [85]:
def hent_stasjonsdata2(stasjon_id):
    """
    Ekstraherer spesifikke felt fra stasjonsdata i JSON-format.

    Args:
        json_data (dict): JSON-data fra API-et.

    Returns:
        dict: Et dictionary med ønskede felt.
    """

    client_id = 'b8b1793b-27ff-4f4d-a081-fcbcc5065b53'
    client_secret = '7f24c0ca-ca82-4ed6-afcd-23e657c2e78c'

    api_url_stasjonsdata = f"https://frost.met.no/sources/v0.jsonld?ids=SN{stasjon_id}&types=SensorSystem"
    
    response_stasjonsdata = requests.get(api_url_stasjonsdata,auth=(client_id,client_secret)) 
    response_stasjonsdata.raise_for_status()
    json_data_stasjonsdata = response_stasjonsdata.json()  # Parse JSON-responsen
    
    # Sjekk om det er data i JSON-strukturen
    data = json_data_stasjonsdata.get("data", [])
   
    if not data:
        return None  # Returner None hvis det ikke er data

    # Hent første element (hvis flere elementer finnes, kan det utvides)
    stasjon = data[0]

    # Ekstraher ønskede felt
    stasjonsinfo = {
        "name": stasjon.get("name"),
        "shortName": stasjon.get("shortName"),
        "masl": stasjon.get("masl"),
        "stationHolder": stasjon.get("stationHolders", []),
        "coordinates": stasjon.get("geometry", {}).get("coordinates", [])
    }

    return stasjonsinfo

In [86]:
print(hent_stasjonsdata2(18700))

{'name': 'OSLO - BLINDERN', 'shortName': 'Oslo (Blindern)', 'masl': 94, 'stationHolder': ['MET.NO'], 'coordinates': [10.72, 59.9423]}
